In [70]:
from interpretable_ssl.pancras.train.train import *

pancras_trainer = PancrasTrainer(split_study=True)

loading data


In [71]:
trainer = pancras_trainer

In [72]:
trainer.batch_size = 32
trainer.hidden_dim = 16
trainer.num_prototypes = 16
model = trainer.get_model()

In [73]:
import torch
model_path = '/home/icb/fatemehs.hashemig/models//pancras/split-study_num-prot-16_hidden-16_bs-32.pth'
model.load_state_dict(torch.load(model_path)["model_state_dict"])
model.to(trainer.device)

ProtClassifier(
  (encoder): VariationalEncoder(
    (linear1): Linear(in_features=4000, out_features=16, bias=True)
    (linear2): Linear(in_features=16, out_features=8, bias=True)
    (linear3): Linear(in_features=16, out_features=8, bias=True)
    (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (decoder): Decoder(
    (linear1): Linear(in_features=8, out_features=16, bias=True)
    (linear2): Linear(in_features=16, out_features=4000, bias=True)
    (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(4000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (classifier): Linear(in_features=16, out_features=14, bias=False)
  (classification_loss): CrossEntropyLoss()
)

In [74]:
list_of_classes = list(range(trainer.dataset.num_classes))

In [75]:
train, test = trainer.dataset.get_train_test()

In [76]:
from torch.utils.data import DataLoader

test_loader = DataLoader(test, batch_size = len(test), shuffle=False)


In [77]:
x, y = next(iter(test_loader))

In [78]:
z, latent, logits = model(x)

In [79]:
preds = logits.argmax(dim=1)

In [96]:
report = classification_report(y.cpu(), preds.cpu(), output_dict=True)

/home/icb/fatemehs.hashemig/miniconda3/envs/scanpy-gpu/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/icb/fatemehs.hashemig/miniconda3/envs/scanpy-gpu/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/icb/fatemehs.hashemig/miniconda3/envs/scanpy-gpu/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 

In [97]:
type(report)

dict

In [98]:
import pandas as pd
df = pd.DataFrame(report)


In [196]:
test_loader = DataLoader(test, batch_size = len(test), shuffle=False)
x_test, y_test = next(iter(test_loader))
for i in range(5):

    _, _, test_logits = model(x_test)
    test_preds = test_logits.argmax(dim=1)

    print((test_preds == y_test).sum())

tensor(804, device='cuda:0')
tensor(831, device='cuda:0')
tensor(817, device='cuda:0')
tensor(807, device='cuda:0')
tensor(817, device='cuda:0')


In [161]:
from torch.utils.data import DataLoader

batch_loader = DataLoader(test, batch_size = 32, shuffle=False)


In [199]:
with torch.inference_mode():
    for i in range(5):
        acc = 0
        cnt = 0
        batch_acc_arr = []
        for x, y in batch_loader:
            z, x_hat, logits = model(x)
            pred = logits.argmax(dim=1)
            # print(pred.shape, y.shape)
            batch_acc = (pred == y).sum()
            batch_acc_arr.append(batch_acc)
            acc += batch_acc
            cnt += len(x)
        print(acc)

tensor(383, device='cuda:0')
tensor(399, device='cuda:0')
tensor(391, device='cuda:0')
tensor(401, device='cuda:0')
tensor(403, device='cuda:0')


In [192]:
x, y = next(iter(batch_loader))
z, x_hat, logits = model(x)
pred = logits.argmax(dim=1)
pred.shape, y.shape, (pred == y).sum()

(torch.Size([32]), torch.Size([32]), tensor(1, device='cuda:0'))

In [198]:
cnt, x_test.shape

(3289, torch.Size([3289, 4000]))

In [67]:
import torch

# Assuming you have a tensor of predicted labels and ground truth labels
predicted_labels = preds
true_labels = y

# Calculate per-class accuracy
per_class_accuracy = []
class_size= []
for class_label in range(num_classes):
    correct = torch.logical_and(predicted_labels == class_label, true_labels == class_label).sum().item()
    total = (true_labels == class_label).sum().item()
    if total == 0:
        per_class_accuracy.append(0.0)
    else:
        per_class_accuracy.append(correct / total)

    class_size.append(total)

print("Per-class accuracy:")
for i, acc in enumerate(per_class_accuracy):
    print(f"Class {i}: {acc}, size: {class_size[i]}")

Per-class accuracy:
Class 0: 0.029880478087649404, size: 502
Class 1: 0.0, size: 109
Class 2: 0.13733075435203096, size: 1034
Class 3: 0.09075907590759076, size: 606
Class 4: 0.691699604743083, size: 253
Class 5: 0.8478632478632478, size: 585
Class 6: 0.0, size: 26
Class 7: 0.0, size: 5
Class 8: 0.203125, size: 128
Class 9: 0.0, size: 16
Class 10: 0.0, size: 7
Class 11: 0.0, size: 13
Class 12: 0.0, size: 5
Class 13: 0.0, size: 0
